### Apply Assumption with Obligor info

In [27]:
from absbox import API,EnginePath

localAPI = API(EnginePath.LOCAL,check=False,lang='english')

Connecting engine server -> http://localhost:8081

✅Connected, local lib:0.28.7, server:0.29.5

#### Assets with obligor info

Let's setup asset with different tags/ids, just plug a map call `obligor` to the asset

    * Asset 1
    
      * ID: "1"
      * Tag:["A","B"]

    * Asset 2
    
      * ID: "2"
      * Tag:["C","B"]

In [28]:
ob1 = {
    "id":"1",
    "tag":["A","B"]
} 

ob2 = {
    "id":"2",
    "tag":["C","B"]
} 


myAsset1 = ["Mortgage"
            ,{"originBalance": 12000.0
             ,"originRate": ["fix",0.045]
             ,"originTerm": 120
             ,"freq": "monthly"
             ,"type": "level"
             ,"originDate": "2021-02-01"
             ,"obligor":ob1
             }
            ,{"currentBalance": 2000.0
             ,"currentRate": 0.075
             ,"remainTerm": 80
             ,"status": "current"}]
myAsset2 = ["Mortgage"
            ,{"originBalance": 12000.0
             ,"originRate": ["fix",0.045]
             ,"originTerm": 120
             ,"freq": "monthly"
             ,"type": "level"
             ,"originDate": "2021-02-01"
             ,"obligor":ob2
             }
            ,{"currentBalance": 5000.0
             ,"currentRate": 0.075
             ,"remainTerm": 80
             ,"status": "current"}]

myPool = {'assets':[myAsset1,myAsset2],
          'cutoffDate':"2022-03-01"}

ppyAssump = (("Mortgage",None ,{"CPR":0.1}, None, None)
             ,None
             ,None)
defAssump = (("Mortgage",{"CDR":0.2} ,None, None, None)
             ,None
             ,None)

#### "ByObligor"

`ByObligor` is introduced `0.29.x` . It accepts a list of (`condition`, `assumptions` ).

#### assumption with obligor id

`ById` will catch assets with its `id` in the list and apply assumptions.

In [29]:
AssetLevelAssumption = ("ByObligor"
                        ,("ById",["1"],ppyAssump)
                        #,("ByTag",["C"],"TagSuperset",defAssump)
                       )

r = localAPI.runPool(myPool
                   ,poolAssump=AssetLevelAssumption
                   ,read=True)

r['PoolConsol'][0].head()

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2024-06-01,7000.00,0.00,0.00,0.00,0,0,0,0.075,None,None,0.00,0.00,0,0,0,0
2024-07-01,6915.22,67.54,43.64,17.24,0,0,0,0.075,None,None,67.54,17.24,0,0,0,0
2024-08-01,6829.94,67.79,43.10,17.49,0,0,0,0.075,None,None,135.33,34.73,0,0,0,0
2024-09-01,6744.73,68.05,42.57,17.16,0,0,0,0.075,None,None,203.38,51.89,0,0,0,0
2024-10-01,6660.12,68.31,42.04,16.30,0,0,0,0.075,None,None,271.69,68.19,0,0,0,0


#### assumption with obligor tag

There are couple way to apply assumption with tags:


* `TagEq` hit when asset tags equals to tags in the assumption

* `TagSubset` hit when asset tags is a subset of the list

* `TagSuperset` hit when asset tags is a superset of the list

* `TagAny` hit when asset tags has any intersetion with tags in assumption

* `("not",<Tag>)`

`TagEq`, only match tags Equals to asset's tag. Here ,there is not match

In [30]:
AssetLevelAssumption = ("ByObligor"
                        ,("ByTag",["C"],"TagEq",ppyAssump)
                       )

r = localAPI.runPool(myPool
                   ,poolAssump=AssetLevelAssumption
                   ,read=True)

r['PoolConsol'][0].head()

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2024-06-01,7000.00,0.00,0.00,0,0,0,0,0.075,None,None,0.00,0,0,0,0,0
2024-07-01,6932.30,67.70,43.75,0,0,0,0,0.075,None,None,67.70,0,0,0,0,0
2024-08-01,6864.16,68.14,43.31,0,0,0,0,0.075,None,None,135.84,0,0,0,0,0
2024-09-01,6795.60,68.56,42.89,0,0,0,0,0.075,None,None,204.40,0,0,0,0,0
2024-10-01,6726.61,68.99,42.46,0,0,0,0,0.075,None,None,273.39,0,0,0,0,0


This will match `Asset 2` 

In [31]:
AssetLevelAssumption = ("ByObligor"
                        ,("ByTag",["C","B"],"TagEq",ppyAssump)
                       )

r = localAPI.runPool(myPool
                   ,poolAssump=AssetLevelAssumption
                   ,read=True)

r['PoolConsol'][0].head()

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2024-06-01,7000.00,0.00,0.00,0.00,0,0,0,0.075,None,None,0.00,0.00,0,0,0,0
2024-07-01,6889.61,67.28,43.48,43.11,0,0,0,0.075,None,None,67.28,43.11,0,0,0,0
2024-08-01,6778.59,67.29,42.77,43.73,0,0,0,0.075,None,None,134.57,86.84,0,0,0,0
2024-09-01,6668.41,67.27,42.09,42.91,0,0,0,0.075,None,None,201.84,129.75,0,0,0,0
2024-10-01,6560.38,67.28,41.41,40.75,0,0,0,0.075,None,None,269.12,170.50,0,0,0,0


`TagAny` will match if any interesection between match tags and asset tags 

In [19]:
AssetLevelAssumption = ("ByObligor"
                        ,("ByTag",["C"],"TagAny",ppyAssump)
                       )

r = localAPI.runPool(myPool
                   ,poolAssump=AssetLevelAssumption
                   ,read=True)

r['PoolConsol'][0].head()

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2024-06-01,7000.00,0.00,0.00,0.00,0,0,0,0.075,None,None,0.00,0.00,0,0,0,0
2024-07-01,6889.61,67.28,43.48,43.11,0,0,0,0.075,None,None,67.28,43.11,0,0,0,0
2024-08-01,6778.59,67.29,42.77,43.73,0,0,0,0.075,None,None,134.57,86.84,0,0,0,0
2024-09-01,6668.41,67.27,42.09,42.91,0,0,0,0.075,None,None,201.84,129.75,0,0,0,0
2024-10-01,6560.38,67.28,41.41,40.75,0,0,0,0.075,None,None,269.12,170.50,0,0,0,0


`TagSubset` will match if asset tags is subset of matching tags

In [20]:
AssetLevelAssumption = ("ByObligor"
                        ,("ByTag",["C"],"TagSubset",ppyAssump)
                       )

r = localAPI.runPool(myPool
                   ,poolAssump=AssetLevelAssumption
                   ,read=True)

r['PoolConsol'][0].head()

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2024-06-01,7000.00,0.00,0.00,0,0,0,0,0.075,None,None,0.00,0,0,0,0,0
2024-07-01,6932.30,67.70,43.75,0,0,0,0,0.075,None,None,67.70,0,0,0,0,0
2024-08-01,6864.16,68.14,43.31,0,0,0,0,0.075,None,None,135.84,0,0,0,0,0
2024-09-01,6795.60,68.56,42.89,0,0,0,0,0.075,None,None,204.40,0,0,0,0,0
2024-10-01,6726.61,68.99,42.46,0,0,0,0,0.075,None,None,273.39,0,0,0,0,0


["C","B"] is a subset of match tag ["C","B"] 

In [21]:
AssetLevelAssumption = ("ByObligor"
                        ,("ByTag",["C","B"],"TagSubset",ppyAssump)
                       )

r = localAPI.runPool(myPool
                   ,poolAssump=AssetLevelAssumption
                   ,read=True)

r['PoolConsol'][0].head()

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2024-06-01,7000.00,0.00,0.00,0.00,0,0,0,0.075,None,None,0.00,0.00,0,0,0,0
2024-07-01,6889.61,67.28,43.48,43.11,0,0,0,0.075,None,None,67.28,43.11,0,0,0,0
2024-08-01,6778.59,67.29,42.77,43.73,0,0,0,0.075,None,None,134.57,86.84,0,0,0,0
2024-09-01,6668.41,67.27,42.09,42.91,0,0,0,0.075,None,None,201.84,129.75,0,0,0,0
2024-10-01,6560.38,67.28,41.41,40.75,0,0,0,0.075,None,None,269.12,170.50,0,0,0,0


`TagSuperset` will match if asset tags is superset of matching tags

In [22]:
AssetLevelAssumption = ("ByObligor"
                        ,("ByTag",["C"],"TagSuperset",ppyAssump)
                       )

r = localAPI.runPool(myPool
                   ,poolAssump=AssetLevelAssumption
                   ,read=True)

r['PoolConsol'][0].head()

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2024-06-01,7000.00,0.00,0.00,0.00,0,0,0,0.075,None,None,0.00,0.00,0,0,0,0
2024-07-01,6889.61,67.28,43.48,43.11,0,0,0,0.075,None,None,67.28,43.11,0,0,0,0
2024-08-01,6778.59,67.29,42.77,43.73,0,0,0,0.075,None,None,134.57,86.84,0,0,0,0
2024-09-01,6668.41,67.27,42.09,42.91,0,0,0,0.075,None,None,201.84,129.75,0,0,0,0
2024-10-01,6560.38,67.28,41.41,40.75,0,0,0,0.075,None,None,269.12,170.50,0,0,0,0


`('not',<TagRule>)` will match with negated

In [26]:
AssetLevelAssumption = ("ByObligor"
                        ,("ByTag",["C"],("not","TagSuperset"),ppyAssump)
                       )

r = localAPI.runPool(myPool
                   ,poolAssump=AssetLevelAssumption
                   ,read=True)

r['PoolConsol'][0].head()

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2024-06-01,7000.00,0.00,0.00,0.00,0,0,0,0.075,None,None,0.00,0.00,0,0,0,0
2024-07-01,6915.22,67.54,43.64,17.24,0,0,0,0.075,None,None,67.54,17.24,0,0,0,0
2024-08-01,6829.94,67.79,43.10,17.49,0,0,0,0.075,None,None,135.33,34.73,0,0,0,0
2024-09-01,6744.73,68.05,42.57,17.16,0,0,0,0.075,None,None,203.38,51.89,0,0,0,0
2024-10-01,6660.12,68.31,42.04,16.30,0,0,0,0.075,None,None,271.69,68.19,0,0,0,0


#### Asset missed by rule

The `ByDefault` will catch every asset not being captured by previous rules

In [25]:
AssetLevelAssumption = ("ByObligor"
                        ,("ByDefault",ppyAssump)
                       )

r = localAPI.runPool(myPool
                   ,poolAssump=AssetLevelAssumption
                   ,read=True)

r['PoolConsol'][0].head()

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2024-06-01,7000.00,0.00,0.00,0.00,0,0,0,0.075,None,None,0.00,0.00,0,0,0,0
2024-07-01,6872.53,67.12,43.37,60.35,0,0,0,0.075,None,None,67.12,60.35,0,0,0,0
2024-08-01,6744.37,66.94,42.56,61.22,0,0,0,0.075,None,None,134.06,121.57,0,0,0,0
2024-09-01,6617.54,66.76,41.77,60.07,0,0,0,0.075,None,None,200.82,181.64,0,0,0,0
2024-10-01,6493.89,66.60,40.99,57.05,0,0,0,0.075,None,None,267.42,238.69,0,0,0,0
